In [1]:
import pandas as pd
import os
import urllib

# Download the data

If `wget` is not installed on your machine, you may want to try `curl URL > file.txt` or the `urllib` package in python.

In [3]:
# %%bash
# mkdir data_ghcn
# cd data_ghcn
# wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
# wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2020.csv.gz
# gzip -d 2020.csv.gz

mkdir: data_ghcn: File exists
--2021-12-01 22:04:02--  https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
正在查找主機 www1.ncdc.noaa.gov (www1.ncdc.noaa.gov)... 205.167.25.167, 205.167.25.171, 205.167.25.172, ...
正在連接 www1.ncdc.noaa.gov (www1.ncdc.noaa.gov)|205.167.25.167|:443... 連上了。
已送出 HTTP 要求，正在等候回應... 200 OK
長度: 10277258 (9.8M) [text/plain]
儲存到：「ghcnd-stations.txt.2」

     0K .......... .......... .......... .......... ..........  0%  255K 39s
    50K .......... .......... .......... .......... ..........  0%  215K 43s
   100K .......... .......... .......... .......... ..........  1% 34.8M 28s
   150K .......... .......... .......... .......... ..........  1% 32.5M 21s
   200K .......... .......... .......... .......... ..........  2%  588K 20s
   250K .......... .......... .......... .......... ..........  2% 16.3M 17s
   300K .......... .......... .......... .......... ..........  3% 15.4M 14s
   350K .......... .......... .......... .......... ..........  3% 39.7M 13

CalledProcessError: Command 'b'mkdir data_ghcn\ncd data_ghcn\nwget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt\nwget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2020.csv.gz\ngzip -d 2020.csv.gz\n'' returned non-zero exit status 1.

# Data Preprocessing

In [90]:
def get_vals(line):
    ls = line.split(',')
    station = ls[0].strip()
    time = ls[1].strip()
    val = float(ls[3])
    return [station, time, val]

def get_stations(filename='data_ghcn/ghcnd-stations.txt'):
    df = pd.read_csv(filename, '/t', header=None)
    df = df[0].str.split(expand=True)[[0, 1, 2, 3]]
    df.columns = ['Station', 'Latitude', 'Longitude', 'Elevation']
    return df


def func(l):
    print(col)
    print(stations.Station.values)
    if col in l:
        v = get_vals(l)
        if v[0] in stations.Station.values:
            return get_vals(l)
    return None

def process_year(year, stations, col='TAVG', basedir='data_ghcn'):
    df_tavg = pd.read_csv(os.path.join(basedir, "%s.csv" % year), header=None)
    df_tavg = df_tavg[(df_tavg[2] == col) & df_tavg[0].isin(stations.Station)]
    display(df_tavg.shape)
    # df_tavg = df_tavg[df_tavg[0].isin(stations.Station)]
    df_tavg = df_tavg[[0, 1, 3]].rename(columns={0: "Station", 1: "Date", 3: col})

    # tavg = []
    # display(stations.Station)
    # with open(os.path.join(basedir, "%s.csv" % year)) as h:
    #     l = h.readline()
    #     while l:
    #         if col in l:
    #             v = get_vals(l)
    #             if v[0] in stations.Station.values:
    #                 tavg.append(get_vals(l))
    #         l = h.readline()
    # df_tavg = pd.DataFrame(tavg, columns=['Station', 'Date', col])
    df_merged = df_tavg.merge(stations, left_on='Station', right_on='Station', how='left')
    df_merged['Date'] = df_merged['Date'].apply(pd.Timestamp)
    for c in ['Latitude', 'Longitude', col, 'Elevation']:
        df_merged[c] = df_merged[c].astype(float)
    return df_merged[['Station', 'Date', col, 'Latitude', 'Longitude', 'Elevation']]

In [91]:
stations = get_stations()
df1 = process_year('2020', stations, col='TAVG')
stations = stations[stations.Station.isin(df1.Station)]
df1

/var/folders/cc/2qxp3wx123b66c1k0ks7g9jw0000gn/T/ipykernel_79319/375170453.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  stations = get_stations()
/Users/rootn/miniconda3/envs/jupyter/lib/python3.10/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Station,Date,TAVG,Latitude,Longitude,Elevation
0,AE000041196,1970-01-01 00:00:00.020200101,211.0,25.333,55.517,34.0
1,AEM00041194,1970-01-01 00:00:00.020200101,217.0,25.255,55.364,10.4
2,AEM00041218,1970-01-01 00:00:00.020200101,199.0,24.262,55.609,264.9
3,AEM00041217,1970-01-01 00:00:00.020200101,205.0,24.433,54.651,26.8
4,AFM00040938,1970-01-01 00:00:00.020200101,54.0,34.210,62.228,977.2
...,...,...,...,...,...,...
2315303,WAM00068006,1970-01-01 00:00:00.020201231,299.0,-17.933,15.983,1100.0
2315304,WAM00068098,1970-01-01 00:00:00.020201231,188.0,-22.983,14.650,88.0
2315305,WF000917530,1970-01-01 00:00:00.020201231,283.0,-13.233,-176.167,27.0
2315306,WFM00091754,1970-01-01 00:00:00.020201231,282.0,-14.317,-178.117,36.0


In [ ]:
df2 = process_year('2020', stations, col='PRCP')

In [ ]:
df = df1.merge(df2[['Station', 'Date', 'PRCP']], on=['Station', 'Date'])
# df.to_csv('data_ghcn/daily_global_weather_2020.csv')
df